In [2]:
import numpy as np
import numpy.linalg as lg 
from scipy.sparse.linalg import gmres
from scipy.linalg import blas
from time import time
from tabulate import tabulate
from  scipy.io import mmread
A1=np.array(mmread('/content/bcsstk20.mtx.gz').todense())
# A1.shape=(485,485) real symmetric positive definite
A2=np.array(mmread('/content/mcca.mtx.gz').todense())
# A2.shape=(180,180) real unsymmetric  
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A4.shape=(817,817) real symmetric positive definite

C1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# C1.shape=(1806, 1806) real symmetric positive definite
C2=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# C2.shape=(3562,3562) real symmetric positive definite
C3=np.array(mmread('/content/bcsstk16.mtx.gz').todense())

B1=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B2=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
B3=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
B4=np.array(mmread('/content/bcsstm26.mtx.gz').todense())


D1=np.array(mmread('/content/impcol_e.mtx.gz').todense())
D2=np.array(mmread('/content/impcol_a.mtx.gz').todense())
D3=np.array(mmread('/content/impcol_b.mtx.gz').todense())
D4=np.array(mmread('/content/impcol_c.mtx.gz').todense())
D5=np.array(mmread('/content/impcol_d.mtx.gz').todense())


E1=np.array(mmread('/content/west0067.mtx.gz').todense())
E2=np.array(mmread('/content/jpwh_991.mtx.gz').todense())
E3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
E4=np.array(mmread('/content/impcol_e.mtx.gz').todense())
E5=np.array(mmread('/content/sherman4.mtx.gz').todense())
E6=np.array(mmread('/content/orsreg_1.mtx.gz').todense())
E7=np.array(mmread('/content/orsirr_1.mtx.gz').todense())
E8=np.array(mmread('/content/add20.mtx.gz').todense())
E9=np.array(mmread('/content/gre__185.mtx.gz').todense())
E10=np.array(mmread('/content/fs_680_3.mtx.gz').todense())
E11=np.array(mmread('/content/fs_680_2.mtx.gz').todense())
E12=np.array(mmread('/content/west0381.mtx.gz').todense())
E13=np.array(mmread('/content/e05r0200.mtx.gz').todense())




# **The GMRES algorithm**

In [2]:
def gmres_iter(A, b,x0,maxiter,restart,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return gmres(A, b, callback=callback,tol = tol,restart=restart,maxiter=maxiter),num_iters

def The_GMRES_algorithm(A, b, x, m, tol, itermax):
    r      = b-blas.dgemv(1.0, A, x)
    beta   = blas.dnrm2(r)   
    ris    = np.zeros(itermax)
    V      = np.zeros((len(b), m+1))
    V[:,0] = r/beta
    H      = np.zeros((m+1,m))
    k      = 0
    while  k < itermax and beta>tol:
        ris[k]   = beta
        for j in range(m):
            w = blas.dgemv(1.0, A, V[:, j])  
            for i in range(j+1):            
                H[i,j]= blas.ddot(w,V[:,i])    
                w     = w - H[i,j]*V[ :,i]
            H[j+1,j] = blas.dnrm2(w)   
            if H[j+1,j]==0:
                break
            V[:, j+1] = w / H[j+1,j] 
        e1      = np.zeros(m+1)
        e1[0]   = 1
        Vm      = V[:,:m]                                         # Vm obtenue en supprimant la dernière colonne de V
        Q, R    = lg.qr(H,mode='complete')                        # application du QR à la matrice H de taille (m+1)*m
        Rm      = R[:m,:m] 
        C       =   np.transpose(Q)                               # matrice extraite en supprimant la dernière colonne de R
        g       = blas.dgemv(beta, C, e1)                         # vecteur de taille m+1
        gm      = g[:m]                                           # vecteur de taille m (en garde les m premières composantes)
        ym      = lg.solve(Rm, gm)
        x       = x + blas.dgemv(1.0,Vm ,ym )      
        r       = b - blas.dgemv(1.0, A, x)
        beta    = blas.dnrm2(r)   
        V[:, 0] = r/beta
        k       = k + 1
    return x, k


# Test:

In [3]:
N=100
m=restart=[20,30,N]
A = 2*np.eye(N) -np.diag(np.ones(N-1),-1)-np.diag(np.ones(N-1),1)
b = np.ones(N)
tol=1e-5
itermax=1000
print('codition umber :',lg.cond(A))

codition umber : 4133.642926801555


In [4]:
my_data = []
for i in range(len(m)):
  l = []
  b=np.ones(N)
  x = np.zeros(N)
  sol  = lg.solve(A, b) 
  itersc = gmres_iter(A,b,x,itermax,restart[i],tol)[1] 
  y=gmres(A,b,x,tol,m[i],itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(A,y)-b)
  x,iterm= The_GMRES_algorithm(A, b, x, m[i], tol, itermax)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(A,x)-b)
  l.append(m[i])
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['m','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid"))

+-----+-------+----------------+-------------+-------------+------------------+-------------+-------------+
|   m |   tol |   Our_nbr_iter |         res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+=====+=======+================+=============+=============+==================+=============+=============+
|  20 | 1e-05 |             74 | 8.51018e-06 | 0.00822569  |             1000 | 9.98424e-05 | 0.0999248   |
+-----+-------+----------------+-------------+-------------+------------------+-------------+-------------+
|  30 | 1e-05 |             32 | 8.87753e-06 | 0.00846728  |              795 | 9.94065e-05 | 0.0971988   |
+-----+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| 100 | 1e-05 |              1 | 3.22662e-12 | 8.67896e-10 |               50 | 5.54502e-12 | 1.25631e-10 |
+-----+-------+----------------+-------------+-------------+------------------+-------------+-------------+


# **Cluster 1:**

In [5]:
matricies=[B1,E2,B2,B3,B4,A3,A4,E5,E6,E7,E8]
tol=1e-5
itermax = 1000
m=restart=30

In [6]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+-------+----------------+-------------+--------------+------------------+-------------+--------------+
| matricies    |   condition number |   tol |   Our_nbr_iter |         res |      Our_err |   scipy_nbr_iter |      sc_res |    scipy_err |
+==============+====================+=======+================+=============+==============+==================+=============+==============+
| (3600, 3600) |            23.7154 | 1e-05 |              1 | 2.28355e-13 |  1.38565e-08 |                3 | 2.87905e-13 |  2.5557e-08  |
+--------------+--------------------+-------+----------------+-------------+--------------+------------------+-------------+--------------+
| (991, 991)   |           142.045  | 1e-05 |              2 | 2.03457e-07 |  1.56593e-06 |               37 | 0.000230357 |  0.000314694 |
+--------------+--------------------+-------+----------------+-------------+--------------+------------------+-------------+--------------+
| (138, 138)   |    

# **Cluster 2:**

In [9]:
matricies=[D1,D2,C3,C1,A1,C2]
tol=1e-5
itermax = 1000
m=restart=30

In [10]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+-------+----------------+----------+------------------+------------------+----------+------------------+
| matricies    |   condition number |   tol |   Our_nbr_iter |      res |          Our_err |   scipy_nbr_iter |   sc_res |        scipy_err |
+==============+====================+=======+================+==========+==================+==================+==========+==================+
| (225, 225)   |        7.10097e+06 | 1e-05 |           1000 | 13.5361  |   1930.39        |             1000 | 13.5361  |   1930.39        |
+--------------+--------------------+-------+----------------+----------+------------------+------------------+----------+------------------+
| (207, 207)   |        1.35164e+08 | 1e-05 |           1000 | 13.4093  | 123241           |             1000 | 13.4093  | 123241           |
+--------------+--------------------+-------+----------------+----------+------------------+------------------+----------+------------------+
| (488

# **Cluster 3:**

In [11]:
matricies=[E2,E5,E6,E7,E8]
tol=1e-5
itermax = 1000
m=restart=30

In [12]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| matricies    |   condition number |   tol |   Our_nbr_iter |         res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+====================+=======+================+=============+=============+==================+=============+=============+
| (991, 991)   |            142.045 | 1e-05 |              2 | 2.03457e-07 | 1.56593e-06 |               37 | 0.000230357 | 0.000314694 |
+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| (1104, 1104) |           2178.63  | 1e-05 |             16 | 6.26214e-06 | 0.000147726 |              333 | 0.000326244 | 0.00867892  |
+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| (2205, 2205) |           6745.27

**Size of Krylov subspace: 20**


In [13]:
matricies=[E2,E5,E6,E7,E8]
tol=1e-5
itermax = 1000
m=restart=20

In [14]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| matricies    |   condition number |   tol |   Our_nbr_iter |         res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+====================+=======+================+=============+=============+==================+=============+=============+
| (991, 991)   |            142.045 | 1e-05 |              3 | 3.19246e-06 | 3.98916e-06 |               46 | 0.000244903 | 0.00126877  |
+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| (1104, 1104) |           2178.63  | 1e-05 |             31 | 8.03502e-06 | 0.000251621 |              478 | 0.000330533 | 0.00897608  |
+--------------+--------------------+-------+----------------+-------------+-------------+------------------+-------------+-------------+
| (2205, 2205) |           6745.27